# First BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100
PREFIXES = ["junkfood_because", "junkfood_but"]

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data = []
for prefix in PREFIXES:
    data_file = f"../data/interim/{prefix}_withprompt.ndjson"

    with open(data_file) as i:
        data += ndjson.load(i)
        
        
label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

data_items = preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
data_items = np.array(data_items)

I0327 14:11:32.511290 139807978702656 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0327 14:11:33.492857 139807978702656 file_utils.py:57] TensorFlow version 2.1.0 available.
I0327 14:11:34.133068 139807978702656 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Training

In [3]:
import torch

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/4*3)
    
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]

    train_dataloader = get_data_loader(train_data, BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, device)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)


I0327 14:11:34.960264 139807978702656 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0327 14:11:34.962032 139807978702656 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd

../quillnlp/models/bert/train.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = softmax(logits.to('cpu'))



Loss history: []
Dev loss: 1.9920349319775899


Epoch:   5%|▌         | 1/20 [00:08<02:36,  8.26s/it]


Loss history: [1.9920349319775899]
Dev loss: 1.5656807323296864


...


Loss history: [2.0948343674341836, 1.3785478621721268, 1.111532673239708, 0.9135365237792333, 0.8139717479546865, 0.7882746507724127, 0.760879397392273, 0.7029429202278455, 0.6822322110335032, 0.69319649040699, 0.6449295605222384, 0.6413164685169855, 0.6772202675541242, 0.6400696138540903, 0.6450052062670389, 0.6769409204522768, 0.5762488270799319, 0.5996729085842768]
Dev loss: 0.5690595532457033




Epoch:  95%|█████████▌| 19/20 [02:35<00:08,  8.12s/it]



Epoch: 100%|██████████| 20/20 [02:43<00:00,  8.04s/it]




Loss history: [2.0948343674341836, 1.3785478621721268, 1.111532673239708, 0.9135365237792333, 0.8139717479546865, 0.7882746507724127, 0.760879397392273, 0.7029429202278455, 0.6822322110335032, 0.69319649040699, 0.6449295605222384, 0.6413164685169855, 0.6772202675541242, 0.6400696138540903, 0.6450052062670389, 0.6769409204522768, 0.5762488270799319, 0.5996729085842768, 0.5690595532457033]
Dev loss: 0.57136228804787
Loading model from /tmp/model.bin


I0327 14:25:07.923154 139807978702656 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0327 14:25:07.924779 139807978702656 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd

## Evaluation

In [4]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

print("Test performance:", precision_recall_fscore_support(all_correct, all_predicted, average="micro"))
print(classification_report(all_correct, all_predicted, target_names=target_names))

Test performance: (0.8677777777777778, 0.8677777777777778, 0.8677777777777778, None)
                                                     precision    recall  f1-score   support

        Unhealthy without Diabetes and Risk Factors       0.98      0.98      0.98       262
                          Diabetes and Risk Factors       1.00      0.99      0.99        94
Nutritional value without Diabetes and Risk Factors       0.89      0.89      0.89        18
                           Obesity without Diabetes       0.89      0.94      0.92        18
                             Unclassified Off-Topic       0.53      0.40      0.46        47
                    School without generating money       0.57      0.40      0.47        52
             Schools providing healthy alternatives       0.94      0.96      0.95       245
                                     Student choice       0.63      0.91      0.74        35
                            Students without choice       0.63      0.71     

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
c = 0
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    assert item.label_id == correct
    c += (item.label_id == predicted)
    print("{}#{}#{}".format(item.text, idx2label[correct], idx2label[predicted]))
    
print()
print(c, "/", len(all_test_data), "=", c/len(all_test_data))

AttributeError: 'BertInputItem' object has no attribute 'label_id'